In [ ]:
!python -V
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.insert(0, os.path.join('..', 'embedding_english'))
from dataset import Dictionary

In [ ]:
pss = Dictionary(os.path.join('..', 'data', 'cmudict-0.7b-simvecs'))
our = Dictionary(os.path.join('..', 'embedding_english', 'simvecs'))
puns = pd.read_csv(os.path.join('..', 'res', 'heterographic_pun_words.txt'), header=None, delimiter=r"\s+")

In [ ]:
print('Pun words', len(puns))
puns['PSSVec'] = puns.apply(lambda row: pss.score(row[0], row[1]), axis=1)
puns['Ours'] = puns.apply(lambda row: our.score(row[0], row[1]), axis=1)
fig = plt.figure(figsize=(10,5), facecolor='w')
sns.distplot(puns['PSSVec'], kde=True, hist=False).set_title('Heterographic Pun Dataset')
sns.distplot(puns['Ours'], kde=True, hist=False).set_xlabel('Cosine Similarity')
fig.legend(labels=['PSSVec', 'Ours'], loc='upper left', bbox_to_anchor=(0.15,0.85))

In [ ]:
puns['diff'] = puns['Ours'] - puns['PSSVec']
# puns['diff'] = puns['PSSVec'] - puns['Ours']
print(puns.sort_values(by=['diff'], ascending=False))

In [ ]:
from dictionary import Dictionary

d = Dictionary(os.path.join('..', 'data', 'cmudict-0.7b'), 'latin1')
def same(w1, w2):
    return d.phones(w1.upper()) == d.phones(w2.upper())

puns = puns[puns.apply(lambda row: not same(row[0], row[1]), axis=1)]
print('Pun words after removing same phoneme words', len(puns))
fig = plt.figure(figsize=(10,5), facecolor='w')
sns.distplot(puns['PSSVec'], kde=True, hist=False).set_title('Distribution of similar sounding words with mean')
sns.distplot(puns['Ours'], kde=True, hist=False).set_xlabel('Cosine Similarity')
fig.legend(labels=['PSSVec', 'Ours'], loc='upper left', bbox_to_anchor=(0.15,0.85))
plt.axvline(puns['PSSVec'].mean(), color=sns.color_palette()[0], linestyle='--')
plt.axvline(puns['Ours'].mean(), color=sns.color_palette()[1], linestyle='--')
# plt.text(best_penalty, 0.5, ' max of average', rotation=0, fontsize=12)